# Selección de Región y Pozos - OilyGigant

## 1. Inicialización

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt

## 2. Definición de variables

In [2]:
data = {'Región 0': 'geo_data_0.csv',
        'Región 1': 'geo_data_1.csv',
        'Región 2': 'geo_data_2.csv'}

# Data para el calculo de ganancias
inversion = 100000000
total_pozos = 200
total_points = 500
revenue_per_unit = 4500
min_pozos = 111.1
max_risk = 2.5

# Bootstrapping
n_bootstraps = 1000

## 3. Funciones

### 3.1 Cargar datos

In [3]:
# Load data
def load_data(path):
    df = pd.read_csv(path)
    # print(df.info())
    return df

- No fué necesario limpiar ni modificar los datos, ya que estos no presentaron valores nulos y estaban en formato correcto (int).

### 3.2 Entrenar y validar modelos: Regresión Lineal

In [4]:
# Entrenamiento y validación de modelo
def train_and_validate(df):
    target = 'product'
    features = df.drop(columns=['id', target], axis=1)
    target = df[target]
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    model = LinearRegression().fit(features_train, target_train)
    predictions = model.predict(features_valid)
    mean = predictions.mean()
    rmse = root_mean_squared_error(target_valid, predictions)
    return target_valid, predictions, mean, rmse

### 3.3 Calcular ganancias

In [5]:
# Calcular ganancias
def profit(predictions):
    top_pozos = pd.Series(predictions).sort_values(
        ascending=False).head(total_pozos)
    revenue_pozos = top_pozos.sum() * revenue_per_unit
    revenue_total = revenue_pozos - inversion
    return revenue_total

- Ingresos totales = Unidades * Ingreso por unidad
- Ganancias = Ingresos totales - Inversión

### 3.4 Bootstrapping

In [6]:
# Bootstrapping: beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas
def bootstrap(predictions):
    rng = np.random.default_rng(12345)
    revenues = []
    for i in range(n_bootstraps):
        subsample = rng.choice(predictions, size=total_points, replace=True)
        revenue_sub = profit(subsample)
        revenues.append(revenue_sub)

    revenues = pd.Series(revenues)
    mean_revenue = revenues.mean()
    cuantil_95 = revenues.quantile(0.95)
    risk = (revenues < 0).mean() * 100
    return mean_revenue, cuantil_95, risk

## 4. Llamado de funciones

### 4.1 Entrenamiento y validación de modelos para cada región

In [7]:
# Entrenamiento y validación de modelos para cada región
models = {}
for region, path in data.items():
    df = load_data(path)
    target_valid, predictions, mean, rmse = train_and_validate(df)
    models[region] = {
        'target_valid': target_valid,
        'predictions': predictions,
        'mean': mean,
        'rmse': rmse
    }

    if mean < min_pozos:
        a = 'menor'
    else:
        a = 'mayor'
    print(f"{region}: RMSE = {rmse:.4f}, Media de predicciones = {mean:.4f} ({a} que la media de {min_pozos} para evitar pérdidas)")

Región 0: RMSE = 37.5794, Media de predicciones = 92.5926 (menor que la media de 111.1 para evitar pérdidas)
Región 1: RMSE = 0.8931, Media de predicciones = 68.7285 (menor que la media de 111.1 para evitar pérdidas)
Región 2: RMSE = 40.0297, Media de predicciones = 94.9650 (menor que la media de 111.1 para evitar pérdidas)


- Todas las regiones tienen una media de predicción menor que 111.1 para evitar pérdidas. Por lo tanto, existe cierto riesgo.
- La región 1 tiene un RMSE muy bajo, por lo que este modelo tiene predicciones más acertadas. Sin embargo, también es el que presenta menor media de predicción.
- Las regiones 0 y 2 tienen medias de predicción más altas, pero también más ruidosas con RMSE alto. Estas predicciones son más variables.

### 4.2 Ganancias por región de los top 200 pozos

In [8]:
# Ganancias por región top 200 pozos
for region, resp in models.items():
    revenue = profit(resp['predictions'])
    resp['revenue'] = revenue
    if revenue < 0:
        b = 'No es rentable invertir en esta región.'
    else:
        b = 'Inversión rentable.'
    print(f"{region}: Ganancias por los 200 pozos más productivos = ${revenue:,.2f}. {b}")

Región 0: Ganancias por los 200 pozos más productivos = $39,960,488.77. Inversión rentable.
Región 1: Ganancias por los 200 pozos más productivos = $24,857,120.52. Inversión rentable.
Región 2: Ganancias por los 200 pozos más productivos = $33,217,543.96. Inversión rentable.


- Las 3 regiones son rentables tomando los 200 pozos más productivos.
- La región 0 tiene mayor beneficio absoluto, y la región 1 el menor.

### 4.3 Bootstrapping para cada región: beneficio promedio, IC 95% y riesgo de pérdidas

In [9]:
# Bootstrapping
for region, resp in models.items():
    mean_revenue, cuantil_95, risk = bootstrap(resp['predictions'])
    resp['mean_revenue'] = mean_revenue
    resp['cuantil_95'] = cuantil_95
    resp['risk'] = risk
    if risk > max_risk:
        c = 'Riesgo alto (>2.5%)'
    else:
        c = 'Riesgo aceptable (<2.5%)'

    print(f"{region}: Beneficio promedio = ${mean_revenue:,.2f}, Cuantil 95% = ${cuantil_95:,.2f}, Riesgo de pérdidas = {risk:.2f}% ({c}).")

Región 0: Beneficio promedio = $3,592,579.42, Cuantil 95% = $5,449,043.28, Riesgo de pérdidas = 0.00% (Riesgo aceptable (<2.5%)).
Región 1: Beneficio promedio = $4,520,765.76, Cuantil 95% = $7,828,286.17, Riesgo de pérdidas = 1.40% (Riesgo aceptable (<2.5%)).
Región 2: Beneficio promedio = $2,707,492.98, Cuantil 95% = $4,279,866.44, Riesgo de pérdidas = 0.50% (Riesgo aceptable (<2.5%)).


- Todas las regiones presentan un riesgo aceptable.
- Con el bootstrapping podemos ver la variabilidad en diferentes escenarios de pozos, acercándonos más a la realidad.
- La región 1 tiene el mayor beneficio promedio, mientras que la región 2 tiene el menor.
- La región 1 tiene el mayor riesgo de perdidas, y la región 0 el menor. Pero ambas por debajo de 2.5%.

## Conclusiones

In [10]:
elegible = {region: resp for region,
            resp in models.items() if resp['risk'] < max_risk}
if not elegible:
    print("No hay regiones elegibles para la inversión (riesgo mayor a 2.5%).")
else:
    best_region = max(elegible, key=lambda x: elegible[x]['mean_revenue'])
    print(f"La mejor región para invertir es: {best_region}")
    print(
        f"Beneficio promedio = ${elegible[best_region]['mean_revenue']:.2f}, Riesgo de pérdidas = {elegible[best_region]['risk']:.2f}%.")

La mejor región para invertir es: Región 1
Beneficio promedio = $4520765.76, Riesgo de pérdidas = 1.40%.


- Se seleccionó la región 1 como la mejor debido a que esta tiene el beneficio promedio más alto y su riesgo de pérdida es menor al 2.5%.